In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# import networkx as nx
import os
import pandas as pd
import time
import json
from csv import writer
import pickle
import seaborn as sb
from power_planner import graphs
from power_planner.utils.utils import get_distance_surface, rescale

In [ ]:
with open("../data/belgium_data_1_2.dat", "rb") as infile:
    (belgium_inst, belgium_edge_inst, belgium_inst_corr, belgium_config) = pickle.load(infile)

### Run once for each instance

In [ ]:
impl_graph_de = graphs.ImplicitLG(belgium_inst, belgium_inst_corr, edge_instance=belgium_edge_inst)
path_impl_de, path_costs_impl_belgium, cost_sum_impl_belgium =  impl_graph_de.single_sp(**vars(belgium_config.graph))


## Make images with one path in each instance

In [ ]:
counter=1
for name, path_impl, graph in zip(["belgium_", "de_", "ch_"],[path_impl_belgium, path_impl_de, path_impl_ch], [impl_graph_belgium, impl_graph_de, impl_graph_ch]):
    plt.figure(figsize=(10,10))
    # plt.subplot(1,3,counter)
    path_impl = np.array(path_impl)
    # path_window = np.array(path_window)
    plt.imshow(np.swapaxes(graph.instance,1,0)) # [20:,20:]
    plt.scatter(path_impl[:,0], path_impl[:,1], label="Optimal path (Cost: 13.8)",c="red", s=3)
    # plt.scatter(path_window[:,1]-50, path_window[:,0]-50, label="Forced through window (14.2)", s=100)
    plt.axis("off")
    # plt.legend(loc="upper left",framealpha=1)
    counter +=1
# plt.tight_layout()
    plt.savefig("../../figure/"+name+"_all_instances.pdf", bbox_inches="tight", pad_inches=0)
    plt.show()

### Construct table about instances

In [ ]:
belgium_inst.shape

In [ ]:
739 * 1300

In [ ]:
start_x = np.where(np.any(belgium_inst_corr!=0, axis=1))[0][0]
end_x = np.where(np.any(belgium_inst_corr!=0, axis=1))[0][-1]
start_y = np.where(np.any(belgium_inst_corr!=0, axis=0))[0][0]
end_y = np.where(np.any(belgium_inst_corr!=0, axis=0))[0][-1]
stripped_inst = belgium_inst[:, start_x:end_x, start_y:end_y]
stripped_corr = belgium_inst_corr[start_x:end_x, start_y:end_y]
print(stripped_inst.shape)

In [ ]:
len(np.where(stripped_corr>0)[0]), len(np.where(stripped_corr>0)[0])/(stripped_corr.shape[0]* stripped_corr.shape[1])


In [ ]:
g = graphs.ImplicitLG(belgium_inst, belgium_inst_corr, edge_instance=belgium_edge_inst)
g.set_shift(belgium_config.graph.start_inds, belgium_config.graph.dest_inds, **vars(belgium_config.graph))

In [ ]:
len(np.where(stripped_corr>0)[0]) * len(g.shifts)

In [ ]:
len(g.shifts), belgium_config.graph.pylon_dist_min, belgium_config.graph.pylon_dist_max

## Run for different graphs

In [ ]:
impl_graph = graphs.ImplicitLG(belgium_inst, belgium_inst_corr, edge_instance=belgium_edge_inst)


In [ ]:
tic = time.time()
belgium_config.graph.ANGLE_WEIGHT=0.3
path_impl, path_costs_impl, cost_sum_impl =  impl_graph.single_sp(**vars(belgium_config.graph))
print(time.time()-tic)

In [ ]:
wg_graph = graphs.WeightedGraph(belgium_inst, belgium_inst_corr)

In [ ]:
tic = time.time()
path_wg, path_costs_wg, cost_sum_wg = wg_graph.single_sp(**vars(belgium_config.graph))
print(time.time()-tic)

In [ ]:
lg_graph = graphs.LineGraph(belgium_inst, belgium_inst_corr)

In [ ]:
tic = time.time()
out_impl = lg_graph.single_sp(**vars(belgium_config.graph))
print(time.time()-tic)

In [ ]:
path_impl, path_costs_impl, cost_sum_impl =  impl_graph.sp_trees(**vars(belgium_config.graph))


## Make plot for Alternative path (force through window figure)

In [ ]:
from power_planner.alternative_paths import AlternativePaths
alt = AlternativePaths(impl_graph)
path_window, path_window_cost, cost_sum_window = alt.path_through_window(
    150, 200, 250, 300
)

In [ ]:
cost_sum_window, cost_sum_impl

In [ ]:
path_impl = np.array(path_impl)
path_window = np.array(path_window)
plt.figure(figsize=(20,10))
plt.imshow(impl_graph.instance[50:,50:])# np.swapaxes( ,1,0))
plt.scatter(path_impl[:,1]-50, path_impl[:,0]-50, label="Optimal path (Cost: 13.8)", s=100)
plt.scatter(path_window[:,1]-50, path_window[:,0]-50, label="Forced through window (14.2)", s=100)
plt.axis("off")
plt.legend(loc="upper left",framealpha=1)
plt.savefig("../../figure/path_through_window.pdf")
plt.show()

## Pipeline figure: take random data and run it

In [ ]:
random_instance = np.random.rand(4, 50,50)
config = belgium_config
start_start = np.array([4,4])
start_dest = np.array([46,46])
PYLON_DIST_MIN = 6
PYLON_DIST_MAX = 10

In [ ]:
scales = [4, 2,1]
dist = [0,5, 5]
corr = np.ones((int(random_instance.shape[1]/scales[0]), int(random_instance.shape[2]/scales[0])))

plt.figure(figsize=(15,10))
for i in range(len(scales)):
    instance = np.array([rescale(img_i, scales[i]) for img_i in random_instance])
    print(instance.shape)
    
    if dist[i]>0:

        # upscale path
        path = (path * scales[i-1]/scales[i]).astype(int)
        # get corridor
        corridor = get_distance_surface(
                instance.shape[1:],
                [path],
                mode="dilation",
                n_dilate=dist[i]
            )
        corr = (corridor > 0).astype(int)
    # plt.imshow(corr)
    # plt.colorbar()
    # plt.show()
    # 
    config.graph.start_inds = (start_start/scales[i]).astype(int)
    config.graph.dest_inds = (start_dest/scales[i]).astype(int)
    config.graph.PYLON_DIST_MIN = PYLON_DIST_MIN/scales[i]
    config.graph.PYLON_DIST_MAX = PYLON_DIST_MAX/scales[i]
    print(config)
    print(instance.shape)
    print(corr.shape)
    graph = graphs.ImplicitLG(instance, corr, verbose=0)
    path, _, _ = graph.single_sp(**vars(config.graph))
    
    # plt.subplot(1, 6, i*2+1)
    # plt.imshow(graph.instance)
    # plt.axis("off")

    
    #overlay_corridor = get_distance_surface(
    #            instance.shape[1:],
    #            [path],
    #            mode="dilation",
    #            n_dilate=dist[i+1]
    #        )
    #
    plt.subplot(1, 3, i+1)
    path = np.array(path)
    plt.imshow(graph.instance)
    # plt.imshow(overlay_corridor, alpha=0.3)
    plt.plot(path[:,1], path[:,0], marker="o", c="white")
    plt.axis("off")
        
plt.savefig("../../figure/pipeline.png")
plt.show()
    

### Random pipeline figure

In [ ]:
overlay_corridor = get_distance_surface(
               instance.shape[1:],
               [path],
               mode="dilation",
               n_dilate=30
           )
overlay_corridor = (overlay_corridor-np.min(overlay_corridor))/(np.max(overlay_corridor)-np.min(overlay_corridor))
arr = np.random.rand(*overlay_corridor.shape)
over = (overlay_corridor) >arr
random = np.sum(
            np.moveaxis(random_instance, 0, -1) * config.graph.class_weights, axis=2
        )
inf_corr = np.absolute(1 - over).astype(float)
inf_corr[inf_corr > 0] = np.inf

graph = graphs.ImplicitLG(random_instance, np.ones(random_instance.shape[1:]), verbose=0)
path_random, _, _ = graph.single_sp(**vars(config.graph))


path_random = np.array(path_random)
plt.imshow(random+inf_corr)
# plt.imshow(overlay_corridor, alpha=0.3)
plt.plot(path_random[:,1], path_random[:,0], marker="o", c="red", linewidth=5, markersize=15)
plt.axis("off")
plt.savefig("../../figure/random_pipeline.png")

plt.imshow(overlay_corridor)
plt.colorbar()
plt.axis("off")
plt.savefig("../../figure/distribution.png")

## Path cost figure

In [ ]:
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#### Necessary function

In [ ]:
from power_planner.utils.utils import normalize
def plot_path_costs(
    instance_in, path, edgecosts, path2, edgecosts2, class_names, out_path=None, buffer=1, strip = 50
):
    """
    Cost visualization: Plot one image for each cost class:
    Arguments:
        instance: n_classes x imgwidth x imgheight sized array
        path: list or array of (x,y) coordinates
        edgecosts: path length x n_classes array or list containing costs
        class_names: n_classes list of names for plot titles
        out_path: where to save
        buffer: num pixels to color (for large images one pixel too small)
    """
    add = 0.4
    
    print(instance_in.shape)
    # wo_zero = instance_in[:, :, np.any(instance_in > 0, axis=(0,1))]
    # instance = wo_zero[:, np.any(wo_zero > 0, axis=(0,2)), :]
    instance = instance_in[:, strip:-strip, strip:-strip]
    path = np.array(path)-strip 
    path2 = np.array(path2)-strip
        
    edgecosts = np.asarray(edgecosts)
    edgecosts2 = np.asarray(edgecosts2)[:, 1:]
    
    print("out costs shape:", edgecosts.shape)
    n_crit = len(instance)
    # exclude angle costs,
    if n_crit < edgecosts.shape[1]:
        edgecosts = edgecosts[:, 1:]

    # iterate over cost classes to make subplots
    plt.figure(figsize=(25, 15))
    fig, axs = plt.subplots(1, n_crit+1, figsize=(25, 15), gridspec_kw={'width_ratios': [9.5 for _ in range(n_crit)] +[1]})
    
    for j in range(n_crit):
        curr_costs = instance[j]
        # from grey scale to colour channel image
        expanded = np.expand_dims(curr_costs, axis=2)
        expanded = np.tile(expanded, (1, 1, 3))
        
        # put values into visible range
        min_c = np.min([np.min(edgecosts[:, j]), np.min(edgecosts2[:, j])])
        max_c = np.max([np.max(edgecosts[1:-1:, j]), np.max(edgecosts2[1:-1, j])])
        normed_env_costs = (edgecosts[:, j]-min_c)/(max_c-min_c)
        normed2 = (edgecosts2[:, j]-min_c)/(max_c-min_c)
        
        
        # colour nodes in path --> cross! --> first input path
        for i, (x, y) in enumerate(path):
            # colour red for high cost
            expanded[x - buffer-1:x + buffer + 2, y-1:y+2] = [normed_env_costs[i]+add, 1 - normed_env_costs[i]+add,0.2]
            expanded[x-1:x+2, y - buffer-1:y + buffer +2] = [normed_env_costs[i]+add, 1 - normed_env_costs[i]+add,0.2]
            
        # colour nodes in path --> circle --> second input path
        for i, (x, y) in enumerate(path2):
            # colour red for high cost
            orig = expanded[x,y].copy()
            expanded[x - buffer:x + buffer + 1, y - buffer:y + buffer +
                     1] = [normed2[i]+add, 1 - normed2[i]+add, 0]
            # expanded[x-buffer+2:x+buffer-1,y-buffer+2:y+buffer-1] = orig
        # comment in next lines for stripping zero rows and columns
        # wo_zero = expanded[:, np.any(curr_costs > 0, axis=0)]
        # wo_zero = wo_zero[np.any(curr_costs > 0, axis=1), :]

        # display
        # ax = plt.subplot(1, n_crit+1, j + 1)
        axs[j].imshow(np.swapaxes(expanded, 1, 0), origin="upper")
        axs[j].set_title(class_names[j], fontsize=30)
        axs[j].axis("off")
        if j==0:
            legend_elements = [mpl.lines.Line2D([0], [0],marker='+', markersize=30, c=[1,0,0], lw=0, label='Normal LCP'),
               mpl.lines.Line2D([0], [0], marker='s', color='g', lw=0, label='Angle-cost LCP',
                      c=[1,0,0], markersize=20)]
            axs[j].legend(handles=legend_elements, loc='upper left', fontsize=30)        
        
    arr = np.zeros((39,2,3))
    for i in range(39):
        arr[i,:] = [i/39+0.4, 1-i/39+0.4, 0.2]
    axs[j+1].imshow(arr, origin="lower")
    axs[j+1].yaxis.tick_right()
    # axs[j+1].yaxis.label_right()
    axs[j+1].set_yticklabels(np.around(np.arange(1.1,-0.1, 0.1),1))
    axs[j+1].set_ylabel('Category-wise costs', fontsize=30)
    axs[j+1].set_xticks([])
    # cmap = plt.get_cmap("RdYlGn")
    # norm = mpl.colors.Normalize(vmin=0, vmax=1)
    # cb1 = mpl.colorbar.ColorbarBase(axs[j+1], cmap=cmap,
    #                                 norm=norm,
    #                                 orientation='vertical')
    # cb1.set_label('Category costs', fontsize=30) 
    
    
    plt.tight_layout()

    # save image
    if out_path is not None:
        plt.savefig(out_path, bbox_inches='tight')
    else:
        plt.show()

#### Compute paths

In [ ]:
impl_graph = graphs.ImplicitLG(belgium_inst, belgium_inst_corr, edge_instance=belgium_edge_inst)
tic = time.time()
belgium_config.graph.ANGLE_WEIGHT=0.3
path_impl, path_costs_impl, cost_sum_impl =  impl_graph.single_sp(**vars(belgium_config.graph))
print(time.time()-tic)
# tune a bit:
tuned_inst = belgium_inst.copy()
for (i,j) in path_impl:
    tuned_inst[:,i-5:i+5, j-5:j+5] =  tuned_inst[:,i-5:i+5, j-5:j+5]+0.3
wg_graph = graphs.WeightedGraph(belgium_inst, belgium_inst_corr)
tic = time.time()
path_wg, path_costs_wg, cost_sum_wg = wg_graph.single_sp(**vars(belgium_config.graph))
print(time.time()-tic)

#### Plot

In [ ]:
plot_path_costs(
    belgium_inst * belgium_inst_corr,
    path_wg,
    path_costs_wg,
    path_impl, path_costs_impl,
    config.graph.layer_classes,
    buffer=3, strip=50,
    out_path="../../figure/path_costs.pdf"
)

#### Find optimal colours

In [ ]:
arr = np.zeros((10,20,3))
for i in range(10):
    arr[i,:] = [i/10+0.4, 1-i/10+0.4, 0.2]
plt.imshow(arr)

#### Save paths (for evaluation plot)

In [ ]:
path_wg_scaled = np.asarray(path_wg) * 2
path_impl_scaled = np.asarray(path_impl) * 2
c_wg = np.sum([impl_graph.instance[i,j] for (i,j) in path_wg])
c_impl = np.sum([impl_graph.instance[i,j] for (i,j) in path_impl])

In [ ]:
with open("test_paths.json", "w") as outfile:
    json.dump([path_wg_scaled.tolist(), path_impl_scaled.tolist()], outfile)

### Random pipeline statistics

#### Ground truth path

In [ ]:
graph = graphs.WeightedGraph(belgium_inst, belgium_inst_corr, verbose=False)
tic = time.time()
path_gt, path_costs_gt, cost_sum_gt = graph.single_sp(**vars(belgium_config.graph))
edges_gt = graph.n_edges
path_groundtruth = np.array(path_gt)
print("number of edges", edges_gt)
time_gt = time.time()-tic

#### Run multiple times to compute statistics

In [ ]:
test_corr = np.ones(belgium_inst_corr.shape)
test_corr[:50,:] = 0
test_corr[:,:50] = 0
test_corr[:,-50:] = 0
test_corr[-50:,:] = 0

### Test code before transformed to script: automatically compute corridor width

In [ ]:
cfg = belgium_config.graph
PIPE1 = [(0.8, 100), (0.5, 50), (0, 0)]
MAX_EDGES = 1000000
D1 = 100
D2 = 0
random = 1

PIPE2 = [(MAX_EDGES, D1), (MAX_EDGES, 0)] # D2), (MAX_EDGES, 0)]
PIPE2 = [(3,50), (2,20), (1,0)] # [(0.9, 50), (0.75, 20), (0,0)] # [1384653, 1426099, 1218601] #

max_nr_edges = []
times_pipeline = []
correct = []
output = []

if random:
    nr_iters = 1 # 00
    mult_factor = 10
    graphclass = graphs.RandomWeightedGraph
else:
    nr_iters = 1
    mult_factor = 13
    graphclass = graphs.WeightedGraph
remain_edges_factor = 1/(PIPE2[0][0])**2

print("predicted edges after first:", remain_edges_factor*edges_gt)

# COMPUTE STATISTICS
for _ in range(nr_iters):

    graph = graphclass(belgium_inst, test_corr, verbose=False) # belgium_inst_corr
    # set shift necessary in case of random graph automatic probability estimation by edge bound
    graph.set_shift(
            cfg.start_inds, cfg.dest_inds,
            **vars(cfg)
        )

    # in each pipeline the corridar is initially everything
    corridor = np.ones(belgium_inst_corr.shape) * 0.5
    edge_numbers = list()

    tic = time.time()

    for pipe,(factor, dist) in enumerate(PIPE2):
        if random:
            factor = 1-(1/factor**2)
            print(factor)
        graph.set_corridor(corridor, cfg.start_inds, cfg.dest_inds, factor_or_n_edges=factor) # , mode="squared")
        
        # prob_arr = np.random.rand(*graph.corridor.shape)
        # prob_arr = (graph.corridor > prob_arr).astype(int)
        # plt.imshow(prob_arr)
        # plt.show()
        
        path_wg = []
        while len(path_wg)==0:
            path_wg, path_costs_wg, cost_sum_wg = graph.single_sp(**vars(cfg))
            graph.remove_vertices(corridor)
        print("actual number edges", graph.n_edges)

        if dist>0:
            corridor = get_distance_surface(
                    graph.hard_constraints.shape,
                    [path_wg],
                    mode="dilation",
                    n_dilate=10 # dist
                )
            # estimated edges are pixels times neighbors divided by resolution squared
            estimated_edges_10 = len(np.where(corridor>0)[0])*len(graph.shifts)/((PIPE2[pipe+1][0])**2)
            print("estimated with distance 10:", estimated_edges_10)
            now_dist = mult_factor * graph.n_edges / estimated_edges_10 
            # print("reduce corridor:", dist) 
            corridor = get_distance_surface(
                    graph.hard_constraints.shape,
                    [path_wg],
                    mode="dilation",
                    n_dilate=int(np.ceil(now_dist))
                )
            print("estimated with distance ", int(np.ceil(now_dist)), len(np.where(corridor>0)[0])*len(graph.shifts)/((PIPE2[pipe+1][0])**2))
        graph.remove_vertices(corridor)
        edge_numbers.append(graph.n_edges)
        # plt.imshow(graph.corridor)
        # plt.colorbar()
        # plt.show()

    time_pipeline = time.time()-tic
    
    times_pipeline.append(time_pipeline)
    max_nr_edges.append(np.max(edge_numbers))
    # correct.append(np.all(np.array(path_wg) == np.asarray(path_gt)))
    output.append([path_wg, path_costs_wg, cost_sum_wg])
    
    # print(np.max(edge_numbers), "equal gt?", np.all(np.array(path_wg) == np.asarray(path_gt)))

    # idee: evaluate per pylon increase in cost
    # log dists

In [ ]:
edge_numbers

### Auswertung

In [ ]:
# MAX_EDGES = 5000000
LEN_PIPE = len(PIPE2)

with open(f"random_results_{MAX_EDGES}_{LEN_PIPE}_{d1}_{d2}.dat", "wb") as outfile:
    pickle.dump((output, max_nr_edges, times_pipeline, correct), outfile)

In [ ]:
from power_planner.utils.utils_ksp import KspUtils

In [ ]:
MAX_EDGES = 1000000
LEN_PIPE = 2
d1 = 100
d2 = 50
with open("../../outputs/random_results_[(4, 50), (1, 0), (1, 0)]", "rb") as outfile:
    (output, max_nr_edges, times_pipeline, correct) = pickle.load(outfile)

In [ ]:
edge_factor = np.mean(max_nr_edges) / edges_gt
print("Reducing the number of edges by a factor of ", edge_factor)
print("Using time ", np.mean(times_pipeline), "compared to", time_gt)
percent_correct = np.around(np.sum(np.array(correct).astype(int))/ len(correct), 2)
print("Percentage correct:", percent_correct)

unique_path_set = []
unique_costs = []
paths_computed = [np.array(o[0]) for o in output]
for (new_path, _, cost) in output:
    already = [len(path)==len(new_path) and np.all(path==new_path) for path in unique_path_set]
    if not np.any(already):
        unique_path_set.append(new_path)
        unique_costs.append(cost)
print("Number of unique produced paths:", len(unique_path_set))
intersection_w_gt = 1-np.mean([np.around(KspUtils.path_distance(path_gt, p2),2) for p2 in paths_computed])
print("Intersections with ground truth", intersection_w_gt)
eucl_w_gt = np.mean([np.around(KspUtils.path_distance(path_gt, p2, "eucl_mean"),2) for p2 in paths_computed])
print("Mean eucledian distance of paths and ground truth", eucl_w_gt)
print("New costs", np.mean(unique_costs), "vs cost gt:", cost_sum_gt)


In [ ]:
res_list = np.around(np.array([np.mean(times_pipeline), edge_factor, percent_correct, np.mean(unique_costs), np.std(unique_costs), intersection_w_gt, eucl_w_gt]), 2).tolist()
list_of_elem = [f"{MAX_EDGES}_{LEN_PIPE}_{d1}_{d2}"] + res_list
with open("random_results.csv", 'a+', newline='') as write_obj:
    # Create a writer object from csv module
    csv_writer = writer(write_obj)
    # Add contents of list as last row in the csv file
    csv_writer.writerow(list_of_elem)

In [ ]:
res_arr = np.array([
    [1000000, 3, 100, 50, 15.5, 0, 0.08, 11.26],
    [1000000, 3, 100, 50, 15.5, 0, 0.08, 11.26]
])
result_table = pd.DataFrame(res_arr)

## Baseline comparison: tests before making script

#### Construct instance where infinity regions have high cost --> only necessary for belgium

In [ ]:
tuned_inst_corr = np.ones(belgium_inst_corr.shape)
tuned_inst = belgium_inst.copy()
inverted = np.absolute(belgium_inst_corr-1).astype("bool")
tuned_inst[:, inverted] = 1

#### run first time: with 8-neighborhood

In [ ]:
graph_bl = graphs.ImplicitLG(tuned_inst, tuned_inst_corr, verbose=True)
# set shift necessary in case of random graph automatic probability estimation by edge bound
# graph_bl.set_shift(cfg.start_inds, cfg.dest_inds, pylon_dist_min=1,
#         pylon_dist_max=1.5,
#         max_angle=cfg.max_angle,
#         max_angle_lg=cfg.max_angle_lg,)
cfg.pylon_dist_min = 1
cfg.pylon_dist_max = 1.5
cfg.edge_weight = 0
path_raster, path_costs_raster, _ = graph_bl.single_sp(**vars(cfg))

In [ ]:
path_raster = np.array(path_raster)
plt.plot(path_raster[:,1], path_raster[:,0])

#### Construct corridor from the raster path

In [ ]:
pylon_spotting_corr = np.zeros(belgium_inst_corr.shape)
for (i,j) in path_raster:
    if belgium_inst_corr[i,j]< np.inf:
        pylon_spotting_corr[i,j] = 1

#### Pylon spotting

In [ ]:
graph_pylon_spotting = graphs.ImplicitLG(belgium_inst, pylon_spotting_corr, edge_instance=belgium_edge_inst, verbose=True)
cfg.pylon_dist_min = 7.5
cfg.pylon_dist_max = 12.5
path_bl,path_cost_bl, cost_sum_bl = graph_pylon_spotting.single_sp(**vars(cfg))


#### Corresponding GT:

In [ ]:
graph_gt = graphs.ImplicitLG(belgium_inst, belgium_inst_corr, edge_instance=belgium_edge_inst, verbose=True)
path_gt,path_cost_gt, cost_sum_gt = graph_gt.single_sp(**vars(cfg))


In [ ]:
path_bl = np.array(path_bl)
path_gt = np.array(path_gt)
plt.plot(path_bl[:,1], path_bl[:,0])
plt.plot(path_gt[:,1], path_gt[:,0])

In [ ]:
print("per pylon cost ", cost_sum_bl/len(path_bl), cost_sum_gt/len(path_groundtruth))

In [ ]:
np.sum(np.array(path_costs_gt) * np.array(cfg.class_weights)), np.sum(np.array(path_cost_bl)[:, 1:] * np.array(cfg.class_weights))

In [ ]:
belgium_config.graph.layer_classes

## Auswertung baseline analysis

In [ ]:
bl_results = pd.read_csv("../../figure/baseline_results_final.csv")

In [ ]:
bl_results["ID"] = ["Baseline" if i%2==0 else "Ours" for i in range(len(bl_results))]

In [ ]:
bl_results

In [ ]:
grouped_res = bl_results.groupby(["instance", "ID"]).agg({"overall time":"mean", "space required for edges":"mean", "angle cost": "mean", 
                                            "weighted sum of costs":"mean",
                                           "mean distance": "max", "max distance":"max"})
grouped_res = np.around(grouped_res, 1)
grouped_res = grouped_res.rename(columns={"overall time": "Time (seconds)", "angle cost": "Angle cost", 
                           "weighted sum of costs": "Resistance", "mean distance": "Average distance from baseline (in meter)",
                          "max distance":"Maximum distance from baseline (in meter)", "space required for edges":"Number of edges"})
grouped_res


In [ ]:
test = bl_results.groupby(["graph", "ID"]).agg({"overall time":"mean", "space required for edges":"mean", "angle cost": "mean", 
                                            "weighted sum of costs":"mean",
                                           "mean distance": "max", "max distance":"max"})
test

#### Plot showing how angle cost change by different weighting

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.barplot(x="angle weight", y="angle cost", hue="ID", data=bl_results)
ax.set_xlabel("Angle weighting", fontsize=15)
ax.set_ylabel("Angle costs", fontsize=15)
ax.legend(fontsize=15)
plt.savefig("../../figure/bar_angle_baseline.png")

## Auswertung random analysis

In [ ]:
from matplotlib.lines import Line2D

In [ ]:
def add_columns_distfactor(res_df):
    pipeline_out = np.zeros((len(res_df), 4))
    # e1, e2, e3, d1,d2 = [],[],[],[],[]
    for i, row in res_df.iterrows():
        pipe_id = (row["ID"]).split("_")
        if len(pipe_id)>1:
            pipe_id = pipe_id[1]
        else:
            pipe_id = pipe_id[0]
        id_split = pipe_id[2:-2].split("), (")
        e1_elem, d1_elem = tuple(id_split[0].split(", "))
        e2_elem, d2_elem = tuple(id_split[1].split(", "))
        pipeline_out[i] = [float(e1_elem), int(d1_elem), float(e2_elem), int(d2_elem)]
        # e1.append(int(e1_elem))
        # e2.append(d2_elem)
    col_name = ["e1", "d1", "e2", "d2"]
    for i in range(4):
        res_df[col_name[i]] = pipeline_out[:,i]
    return res_df

#### Different loading functions

In [ ]:
random = pd.read_csv("../../outputs/random_results_2908.csv") # random_results_prior.csv")# [:-75] #
deterministic = pd.read_csv("../../outputs/nonrandom_pipelines.csv") # andom_results_prior.csv") # 
prior = pd.read_csv("../../outputs/random_results_prior.csv") [75:]
# random = res_df[res_df["ID"].str.contains("0000")]

In [ ]:
in_df = pd.read_csv("../../outputs/random_results_0109.csv")
deterministic = in_df[~in_df["ID"].str.contains("\[0.")]
random = in_df[in_df["ID"].str.contains("\[0.")]

In [ ]:
in_df = pd.read_csv("../../outputs/random_results_final_0209.csv")

In [ ]:
# PIPELINE LENGTH
pipeline_length = np.array([len(val.split(",")) for val in in_df["ID"].values])
np.unique(pipeline_length)
random = in_df[pipeline_length==2]
deterministic = in_df[pipeline_length==4]
prior = in_df[pipeline_length==6]

In [ ]:
random = in_df[in_df["prior"]=="no prior"]
prior = in_df[in_df["prior"]=="prior"]
deterministic = in_df[in_df["prior"]=="simple downsampling"]
watershed = in_df[in_df["prior"]=="watershed"]
print(len(random), len(prior), len(deterministic), len(watershed))

In [ ]:
#random = add_columns_distfactor(random)
# deterministic = add_columns_distfactor(deterministic)
# prior = add_columns_distfactor(prior)

In [ ]:
from scipy.stats import pearsonr
random = in_df[in_df["ID"].str.contains("\[0.")]
# pearsonr(np.arange(30), [0 for _ in range(15)] + [1 for _ in range(15)])
pearsonr(random["time"], random["average intersection with gt"])

#### Main plot

In [ ]:
plot_x_axis = "remaining edges" # "time" # 
plot_y_axis = "average intersection with gt" # "average costs" # " percent correct" #  
# ' average eucledian distance from GT' #  #  # 
markers = ["o", "x", "+"]
plt.figure(figsize=(11,7))
for ma,inst in zip(markers, ["belgium", "de", "ch"]): # , "ch"]):
    random_inst = random[random["instance"]==inst]
    deterministic_inst = deterministic[deterministic["instance"]==inst]
    prior_inst = prior[prior["instance"]==inst]
    watershed_inst = watershed[watershed["instance"]==inst]
    # random_res = random_de[random_de["e1"]>10]
    plt.scatter(watershed_inst[plot_x_axis], watershed_inst[plot_y_axis], marker=ma, c="orange", s=100, alpha=0.5)
    # static_res = all_dfs # random_de[random_de["e1"]<10]
    plt.scatter(deterministic_inst[plot_x_axis], deterministic_inst[plot_y_axis], marker=ma, c="red", s=100, alpha=0.5)
    plt.scatter(random_inst[plot_x_axis], random_inst[plot_y_axis], marker=ma, c="blue", s=100, alpha=0.5)
    plt.scatter(prior_inst[plot_x_axis], prior_inst[plot_y_axis], marker=ma, c="green", alpha=0.5, s=100)
legend_elements = [Line2D([0], [0], color='b', marker='o', lw=10, label='Random Pipeline'),
                   Line2D([0], [0], color='g', marker='o', lw=10, label='Random with prior'),
                   Line2D([0], [0], marker='o', color='r', lw=10, label='Deterministic Pipeline'),
                   Line2D([0], [0], color='orange', marker='o', lw=10, label='Deterministic with watershed'),
                  Line2D([0], [0], marker='o', markersize=10, color='black', lw=0, label='Instance 1'),
                  Line2D([0], [0], marker='x', markersize=10, color='black', lw=0, label='Instance 2'),
                  Line2D([0], [0], marker='+', markersize=10, color='black', lw=0, label='Instance 3')]
if plot_y_axis=="average intersection with gt":
    plt.ylim(0,1.1)
    ylabel = "Average IoU with LCP"
    name="intersection"
elif plot_y_axis=="average costs":
    # legend=plt.legend(handles=legend_elements, loc='lower left', fontsize=15, framealpha=0)
    # legend.get_frame().set_linewidth(3)
    name="costs"
    ylabel = "Average path costs"
elif plot_y_axis==" percent correct":
    legend=plt.legend(handles=legend_elements, loc='center right', fontsize=15)
    legend.get_frame().set_linewidth(3)
    name="percent_correct"
    ylabel = "Percent correct (LCP found)"
if plot_x_axis=="remaining edges":
    plt.xlim(0,0.4)
    plt.xlabel("Ratio of remaining edges", fontsize=22, weight="bold")
else:
    # plt.xlim(0,26)
    name=name+"_time"
    plt.xlabel("Runtime (in seconds)", fontsize=22, weight="bold") 
    # plt.xscale("log")
plt.ylabel(ylabel, fontsize=22, weight="bold")
# plt.savefig("../../figure/"+name+".pdf", bbox_inches="tight", pad_inches=0.2)
plt.show()

In [ ]:
factor_choice = [2,3,4,5]
for factor in [0.1, 0.15, 0.2, 0.25]:
    best_factor = factor_choice[np.argmin([np.abs(1/f**2 - factor) for f in factor_choice])]

In [ ]:
# backup code
PIPELINES = []
for factor1 in [2, 3, 4, 5]:
    for factor2 in [1, 2]:
        if factor1 <= factor2:
            continue
        if factor2 == 1:
            PIPELINES.append([factor1, factor2])
        else:
            PIPELINES.append([factor1, factor2, 1])
print("PIPELINES:", PIPELINES)


#### Merge serveral csvs

In [ ]:
dfs = []
for infile in ["2708", "debech","50", "cluster_backup"]:
    dfs.append(pd.read_csv("../../outputs/random_results_"+infile+".csv"))
all_dfs = pd.concat(dfs)

In [ ]:
all_dfs = all_dfs[all_dfs["ID"].str.contains("(1, 0)")]

In [ ]:
all_dfs.to_csv("../../outputs/nonrandom_pipelines.csv")

In [ ]:
first_part = pd.read_csv("../../outputs/random_results_0109.csv")
second_part = pd.read_csv("../../outputs/random_final_0209.csv")

In [ ]:
first_part["prior"] = "simple downsampling"

In [ ]:
first_part.loc[first_part["ID"].str.contains("\[0."), "prior"] = "no prior"

In [ ]:
all_dfs = pd.concat([first_part, second_part])
all_dfs.to_csv("random_results_final_0209.csv")

## Diverse Ksp

In [ ]:
with open("compare_diverse_thesis_belgium.dat", "rb") as outfile:
    all_ksps = pickle.load(outfile)
path_csv = pd.read_csv("compare_diverse_thesis_belgium.csv")

In [ ]:
def plot_k_sp(ksp, inst, out_path=None):
    """
    Plot k shortest paths on the instance
    Arguments:
        ksp: list of infos for the k shortest path: for each path, the first
            entry is the path itself, the second the costs array, the third
            the cost sum
        inst: instance to plot on
    """
    # get relevant information
    costs = [k[2] for k in ksp]
    paths = [k[0] for k in ksp]

    # plot main image (cost surface)
    plt.figure(figsize=(10, 20))
    plt.imshow(np.swapaxes(inst, 1, 0))
    # iterate over k shortest paths
    for i, path in enumerate(paths):
        path = np.asarray(path)
        plt.scatter(
            path[:, 0], path[:, 1], label=str(round(costs[i], 2)), s=50
        )
    # plot and save
    leg = plt.legend(fontsize=15)
    leg.set_title('Costs', prop={'size': 15})
    # plt.title(out_path.split("/")[1], fontsize=20)
    plt.tight_layout()
    plt.axis("off")

### final code to plot all paths

In [ ]:
cutoff = 55
disp_inst = np.swapaxes(impl_graph.instance, 1,0)[cutoff:-cutoff, cutoff:-cutoff]
for name, df_grouped in path_csv.groupby("name"):
    fig = plt.figure(figsize=(15,10))
    l = len(df_grouped)
    sorted_ind = np.array(list(df_grouped.index))[np.argsort(df_grouped["threshold"])]
    for i, ind in enumerate(sorted_ind):
        plt.subplot(1,l+1, i+1)
        plt.imshow(disp_inst)
        
        # preparation
        ksp = all_ksps[ind]
        costs = [k[2] for k in ksp]
        paths = [k[0] for k in ksp]

        # plot main image (cost surface)
        # iterate over k shortest paths
        for i, path in enumerate(reversed(paths)):
            path = np.asarray(path)-cutoff
            plt.scatter(
                path[:, 0], path[:, 1], label=str(round(costs[i], 2)), s=50
            )
        # plot and save
        # leg = plt.legend(fontsize=15)
        # leg.set_title('Costs', prop={'size': 15})
    
        plt.axis("off")
        plt.title(df_grouped.loc[ind, "threshold"], fontsize=20)
    # fig.suptitle(name, fontsize=20)
    fig.tight_layout()
    # fig.subplots_adjust(top=0.88)
    plt.savefig("../../figure/ksp_"+name+".pdf", bbox_inches = 'tight',pad_inches = 0)
    plt.show()

## Ksp time plot (runtime by diversity

In [ ]:
import matplotlib 
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)

font = {'size'   : 20}

matplotlib.rc('font', **font)

In [ ]:
ksp_res = pd.read_csv("compare_diverse_thesis_belgium2.csv")

In [ ]:
plt.figure(figsize=(10,10))
for name, grouped in ksp_res.groupby("name"):
    if "set" in name:
        sorted_grouped = grouped.sort_values(by="threshold", ascending=False)
    else:
        sorted_grouped = grouped.sort_values(by="threshold")
    if "dispersion" in name: # or "mean" in name:
        print(grouped["times"].values)
        continue
    plt.plot(np.arange(len(grouped))/(len(grouped)-1), sorted_grouped["times"].values, label=name, linewidth=3)
plt.legend()
plt.yscale("log")
plt.xticks([])
plt.xlabel("Diversity threshold", weight="bold")
plt.ylabel("Runtime (log scale)", weight="bold")
plt.savefig("../../figure/ksp_times.pdf")
plt.show()

## Graph comparison auswertung

In [ ]:
graph_comp = pd.read_csv("../../outputs/graph_comp_0109.csv")

In [ ]:
graph_comp

In [ ]:
grouped_res = graph_comp.groupby(["instance", "resolution", "graph"]).agg({"overall time":"mean", "space required for edges":"mean", "angle cost": "mean", 
                                            "weighted sum of costs":"mean"})
grouped_res = np.around(grouped_res, 1)
grouped_res = grouped_res.rename(index={"belgium":"Instance 1", "ch":"Instance 3", "de":"Instance 2"}, columns={"overall time": "Time (seconds)", "graph":"Shortest path implementation", "angle cost": "Angle cost", 
                           "weighted sum of costs": "Resistance", "space required for edges":"Space required for graph (~number of edges)"})
grouped_res = grouped_res.sort_values(by=["instance", "resolution"])
grouped_res


In [ ]:
grouped_res.to_csv("../../figure/graph_comp.csv")

#### Plot comparisons

In [ ]:
import seaborn as sns

In [ ]:
resetted = grouped_res.reset_index()

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.barplot(x="instance", y="Angle cost", hue="graph", data=resetted)
ax.set_ylabel("Angle cost", weight="bold", fontsize=25)
ax.set_xlabel("Instance", weight="bold", fontsize=25)
ax.set_xticklabels(["1","2","3"])
# ax.set_xlabel("")
plt.savefig("../../figure/graphcomp_anglecost.pdf", bbox_inches="tight", pad_inches=0.2)

In [ ]:

plt.figure(figsize=(10,6))
ax = sns.barplot(x="resolution", y="Space required for graph (~number of edges)", hue="graph", data=resetted, ci=None)
ax.set_ylabel("Number of edges", weight="bold", fontsize=25)
ax.set_xlabel("Resolution (in m)", weight="bold", fontsize=25)
plt.yscale("log")
plt.legend([],[], frameon=False)
plt.savefig("../../figure/graphcomp_nedges.pdf", bbox_inches="tight", pad_inches=0.2)

# Pipeline evaluation 2: pipeline vs direct

In [ ]:
pipes_df = pd.read_csv("../../outputs/pipeline_eval.csv")

In [ ]:
# pipes_df_test = pipes_df[pipes_df["graph"]=="Implicit line graph"]
plt.figure(figsize=(10,6))
ax = sns.barplot(x="instance", y="overall time", hue="ID", data=pipes_df, ci=None) # space required for edges
ax.set_ylabel("Runtime (s)", weight="bold", fontsize=25) # Max. number of edges
ax.set_xlabel("Instance (resolution)", weight="bold", fontsize=25)
# plt.yscale("log")
ax.set_xticklabels(["1 (10m)","2 (20m)","3 (20m)"])
plt.legend(title="Pipeline", ncol=2)
# plt.savefig("../../figure/pipeline_eval_time.pdf", bbox_inches="tight", pad_inches=0.2)
plt.show()

In [ ]:
# FOR TESTING
full = pipes_df[pipes_df["ID"]=="[1]"]
less = pipes_df[pipes_df["ID"]!="[1]"]
full_normal = full[full["graph"]=="Normal graph"]
full_impl = full[full["graph"]=="Implicit graph"]

In [ ]:
grouped_res = pipes_df.groupby(["instance", "graph", "ID"]).agg({"overall time":"mean", "space required for edges":"mean", "angle cost": "mean", 
                                            "weighted sum of costs":"mean"})
grouped_res = np.around(grouped_res, 1)
grouped_res = grouped_res.rename(index={"belgium":"Instance 1", "ch":"Instance 3", "de":"Instance 2"})
            #, columns={"overall time": "Time (seconds)", "graph":"Shortest path implementation", "angle cost": "Angle cost", 
                          # "weighted sum of costs": "Resistance", "space required for edges":"Space required for graph (~number of edges)"})
grouped_res = grouped_res.sort_values(by=["instance", "graph"])
grouped_res

In [ ]:
10760412 / 139824516

In [ ]:
normalized_grouped = grouped_res.groupby(["instance", "graph"]).transform(lambda x: (x/x.iat[0])) # , axis=0) # map(x["angle cost"][0], x)) #


In [ ]:
agg_normed = normalized_grouped.groupby("ID").agg({"overall time":["mean", "std"], "weighted sum of costs":["mean", "std"], 
                                                   "space required for edges":["mean", "std"]})
agg_normed


### Normalized plot

In [ ]:
barWidth = 0.25
# set height of bar
columns = ["overall time", "weighted sum of costs", "space required for edges"]
names = ["Runtime (s)", "Cost", "Max. number of edges"]
colours = ["blue", "red", "green"]
r1 = np.arange(len(agg_normed)).astype("float")
plt.figure(figsize=(10,6))
i = 0
for colour, column, name in zip(colours, columns, names):
    # Make the plot
    values = agg_normed[column, "mean"]
    plt.bar(r1+i*barWidth, values, color=colour, width=barWidth, edgecolor='white', label=name, yerr=agg_normed[column, "std"])
    i+=1
plt.xlabel('Pipeline', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(agg_normed))], agg_normed.index)
plt.ylabel("Percent of optimal path", weight="bold")
plt.ylim(0,1.3)
# Create legend & Show graphic
plt.legend(ncol=3, fontsize=15, loc="upper center")
plt.savefig("../../figure/pipeline_eval_allinone.pdf")
plt.show()
